In [41]:
import random as rn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [42]:
X = np.load("dataset/data.npy")
labels = np.load("dataset/label.npy")

In [43]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(labels)
y = to_categorical(y, 10)

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

In [45]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

x_train shape:  (2500, 150, 150, 3)
x_test shape:  (626, 150, 150, 3)
y_train shape:  (2500, 10)
y_test shape:  (626, 10)


In [48]:
# know neural ntwrk architecture # work in sequential model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation
from keras.optimizers import Adam

In [49]:
# modelling 
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size=(5,5), padding = "Same", activation="relu", input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding  = "Same", activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3), padding  = "Same", activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3), padding  = "Same", activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(10, activation="softmax"))

In [50]:
# preprocess
from keras.preprocessing.image import ImageDataGenerator
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,   # set each sample mean to 0
    featurewise_std_normalization=False,   # divide inputs by std of the dataset
    samplewise_std_normalization= False,   # divide each input by its std
    zca_whitening=False,   # dimesion reduction
    rotation_range=10,    # randomly rotate images in the range 10 degrees
    zoom_range=0.1,      # Randomly zoom image 10%
    width_shift_range=0.2,   # randomly shift images horizontally 20%
    height_shift_range=0.2,   # randomly shift images vertically 20%
    horizontal_flip=True,     # randomly flip images
    vertical_flip=False    # randomly flip images
)
datagen.fit(x_train)

In [51]:
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics = ["accuracy"])

In [52]:
# Train the model
batch_size = 128
epochs = 50

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.1)

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test))

Epoch 1/50
19/19 [==============================] - 24s 1s/step - loss: 16.7708 - accuracy: 0.1336 - val_loss: 2.1078 - val_accuracy: 0.2109
Epoch 2/50
19/19 [==============================] - 23s 1s/step - loss: 2.0417 - accuracy: 0.2255 - val_loss: 2.0463 - val_accuracy: 0.2556
Epoch 3/50
19/19 [==============================] - 25s 1s/step - loss: 1.9098 - accuracy: 0.2976 - val_loss: 1.8027 - val_accuracy: 0.3514
Epoch 4/50
19/19 [==============================] - 25s 1s/step - loss: 1.7915 - accuracy: 0.3402 - val_loss: 1.8716 - val_accuracy: 0.3371
Epoch 5/50
19/19 [==============================] - 25s 1s/step - loss: 1.6615 - accuracy: 0.3963 - val_loss: 1.6444 - val_accuracy: 0.3930
Epoch 6/50
19/19 [==============================] - 25s 1s/step - loss: 1.5876 - accuracy: 0.4262 - val_loss: 1.6972 - val_accuracy: 0.3770
Epoch 7/50
19/19 [==============================] - 26s 1s/step - loss: 1.5827 - accuracy: 0.4300 - val_loss: 1.6496 - val_accuracy: 0.4281
Epoch 8/50
19/19 [=

In [53]:
test_loss, test_acc = model.evaluate(x_test, y_test)

20/20 [==============================] - 2s 83ms/step - loss: 1.2317 - accuracy: 0.6406


In [54]:
# Save the model
#model.save("model/flower-recognition-model.h5")
# Save the model
model.save("model/flower-recognition-model-50-epochs.h5")